In [1]:
from google.colab import drive
import os  # Add this import statement

drive.mount('/content/drive')

base_path = '/content/drive/MyDrive/Neural_Style_Transfer'
content_image_dir = f'{base_path}/content_image'
style_image_dir = f'{base_path}/style_image'
output_dir = f'{base_path}/output'

os.makedirs(output_dir, exist_ok=True)  # Now this will work


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch torchvision pillow imageio

In [3]:
import torch
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image
import numpy as np
import os
import imageio

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
def preprocess_image(image_path, transform):
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0).to(device)

def postprocess(tensor):
    image = tensor.to('cpu').clone().squeeze(0)
    image = image.detach().numpy().transpose(1, 2, 0)
    image = image * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    image = np.clip(image, 0, 1)
    return (image * 255).astype(np.uint8)

def gram_matrix(input_tensor):
    batch_size, c, h, w = input_tensor.size()
    features = input_tensor.view(batch_size * c, h * w)
    G = torch.mm(features, features.t())
    return G.div(batch_size * c * h * w)

def get_features(image, model, layer_indices):
    features = []
    x = image
    for idx, layer in enumerate(model):
        x = layer(x)
        if idx in layer_indices:
            features.append(x)
    return features


In [5]:
# Configuration
content_path = f"{content_image_dir}/content1.jpeg"
style_path = f"{style_image_dir}/style3.jpeg"
iterations = 1000
content_weight = 1e2
style_weight = 1e7
lr = 0.05

# Transform
transform = transforms.Compose([
    transforms.Resize(512),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load images
content = preprocess_image(content_path, transform)
style = preprocess_image(style_path, transform)

# Load VGG19
vgg = models.vgg19(pretrained=True).features.to(device).eval()
for param in vgg.parameters():
    param.requires_grad_(False)

content_layers = [22]
style_layers = [1, 6, 11, 20, 29] # Keep these for now, but consider experimenting

content_features = get_features(content, vgg, content_layers)
style_features = get_features(style, vgg, style_layers)
style_grams = [gram_matrix(f) for f in style_features]

# Initial image
target = content.clone().requires_grad_(True)
optimizer = optim.LBFGS([target], lr=lr)



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
intermediate_images = []
iteration = [0]

print('Starting optimization...')

while iteration[0] <= iterations:
    def closure():
        optimizer.zero_grad()
        target_content = get_features(target, vgg, content_layers)
        target_style = get_features(target, vgg, style_layers)
        target_grams = [gram_matrix(layer) for layer in target_style]

        content_loss = content_weight * torch.mean((target_content[0] - content_features[0])**2)
        style_loss = 0
        for tg, sg in zip(target_grams, style_grams):
            style_loss += torch.mean((tg - sg)**2)
        style_loss *= style_weight
        total_loss = content_loss + style_loss
        total_loss.backward()

        if iteration[0] % 100 == 0:
            print(f"Iteration {iteration[0]} | Content Loss: {content_loss.item():.2f}, Style Loss: {style_loss.item():.2f}")
            with torch.no_grad():
                img = postprocess(target)
                intermediate_images.append(img)
                Image.fromarray(img).save(os.path.join(output_dir, f'iter_{iteration[0]}.png'))
        iteration[0] += 1
        return total_loss
    optimizer.step(closure)

print('Optimization complete!')

Starting optimization...
Iteration 0 | Content Loss: 0.00, Style Loss: 1333.08
Iteration 100 | Content Loss: 93.34, Style Loss: 903.43
Iteration 200 | Content Loss: 105.59, Style Loss: 682.08
Iteration 300 | Content Loss: 102.70, Style Loss: 535.90
Iteration 400 | Content Loss: 100.42, Style Loss: 421.82
Iteration 500 | Content Loss: 97.83, Style Loss: 321.12
Iteration 600 | Content Loss: 94.30, Style Loss: 247.65
Iteration 700 | Content Loss: 90.68, Style Loss: 200.95
Iteration 800 | Content Loss: 87.31, Style Loss: 171.93
Iteration 900 | Content Loss: 84.29, Style Loss: 153.37
Iteration 1000 | Content Loss: 81.90, Style Loss: 140.97
Optimization complete!


In [7]:
# Save final image
final_image = postprocess(target)
Image.fromarray(final_image).save(os.path.join(output_dir, 'final.jpg'))

# Save progress GIF
if intermediate_images:
    imageio.mimsave(os.path.join(output_dir, 'progress.gif'),
                    [Image.fromarray(img) for img in intermediate_images], duration=500, loop=0)
print("Saved final outputs to:", output_dir)

Saved final outputs to: /content/drive/MyDrive/Neural_Style_Transfer/output


In [8]:
#print the above output image in this notebook

from IPython.display import Image as IPythonImage

# Display the final generated image
final_image_path = os.path.join(output_dir, 'final.jpg')
display(IPythonImage(filename=final_image_path))

# Display the progress GIF
if intermediate_images:
    progress_gif_path = os.path.join(output_dir, 'progress.gif')
    display(IPythonImage(filename=progress_gif_path))

Output hidden; open in https://colab.research.google.com to view.